In [14]:
import os
import sys
import pandas as pd
import numpy as np

In [15]:
input_data_folder = "../input_annotations"
in_filename_csv = "test_log.csv"

In [16]:
col_list = ["caseid", "task", "end_timestamp"]

In [17]:
caseid_output = pd.read_csv(caseid_output_route, usecols=col_list)

NameError: name 'caseid_output_route' is not defined

In [18]:
, usecols=col_list

'usecols=col_list'

In [19]:
data = pd.read_csv(os.path.join(input_data_folder, in_filename_csv), sep=",")

In [23]:
print(data[2:])

     caseid              task user              end_timestamp    role  \
2       CNA  ER Sepsis Triage    A  2014-09-19 06:28:31+00:00  Role 2   
3       CNA        Leucocytes    B  2014-09-19 06:34:00+00:00  Role 4   
4       CNA               CRP    B  2014-09-19 06:34:00+00:00  Role 4   
5        DG   ER Registration    A  2014-09-19 06:51:17+00:00  Role 2   
6        DG         ER Triage    C  2014-09-19 06:57:48+00:00  Role 5   
...     ...               ...  ...                        ...     ...   
3765     QK               CRP    B  2015-03-02 07:00:00+00:00  Role 4   
3766     QK               CRP    B  2015-03-04 07:00:00+00:00  Role 4   
3767     QK        Leucocytes    B  2015-03-06 07:00:00+00:00  Role 4   
3768     QK               CRP    B  2015-03-06 07:00:00+00:00  Role 4   
3769     QK         Release A    E  2015-03-07 10:00:00+00:00  Role 6   

           dur  dur_norm  ac_index  rl_index  
2         28.0  0.235192         5         2  
3        329.0  0.405044     

In [13]:
data = pd.read_csv(os.path.join(input_data_folder, in_filename_csv), sep=",", usecols=col_list, dtype={'user': str})

NameError: name 'pd' is not defined

In [16]:
print(data[:2])

  caseid             task
0    CNA  ER Registration
1    CNA        ER Triage


In [ ]:
def get_csv_events_data(data):
    """
    reads and parse all the events information from a csv file
    """
    log = pd.read_csv(input, dtype={'user': str})
    if one_timestamp:
        column_names['Complete Timestamp'] = 'end_timestamp'
        log = log.rename(columns=column_names)
        log = log.astype({'caseid': object})
        log = (log[(log.task != 'Start') & (log.task != 'End')]
               .reset_index(drop=True))
        if filter_d_attrib:
            log = log[['caseid', 'task', 'user', 'end_timestamp']]
        log['end_timestamp'] = pd.to_datetime(log['end_timestamp'],
                                              format=timeformat)
    else:
        column_names['Start Timestamp'] = 'start_timestamp'
        column_names['Complete Timestamp'] = 'end_timestamp'
        log = log.rename(columns=column_names)
        log = log.astype({'caseid': object})
        log = (log[(log.task != 'Start') & (log.task != 'End')]
               .reset_index(drop=True))
        if filter_d_attrib:
            log = log[['caseid', 'task', 'user',
                       'start_timestamp', 'end_timestamp']]
        log['start_timestamp'] = pd.to_datetime(log['start_timestamp'],
                                                format=timeformat)
        log['end_timestamp'] = pd.to_datetime(log['end_timestamp'],
                                              format=timeformat)
    log['user'].fillna('SYS', inplace=True)
    data = log.to_dict('records')
    append_csv_start_end()
    split_event_transitions()

In [31]:
import itertools as it

In [53]:
def append_csv_start_end(data):
    new_data = list()
    data = sorted(data, key=lambda x: x['caseid'])
    for key, group in it.groupby(data, key=lambda x: x['caseid']):
        trace = list(group)
        for new_event in ['Start', 'End']:
            idx = 0 if new_event == 'Start' else -1
            t_key = 'end_timestamp'
            if not one_timestamp and new_event == 'Start':
                t_key = 'start_timestamp'
            temp_event = dict()
            temp_event['caseid'] = trace[idx]['caseid']
            temp_event['task'] = new_event
            temp_event['user'] = new_event
            temp_event['end_timestamp'] = trace[idx][t_key]
            if not one_timestamp:
                temp_event['start_timestamp'] = trace[idx][t_key]
            if new_event == 'Start':
                trace.insert(0, temp_event)
            else:
                trace.append(temp_event)
        new_data.extend(trace)
    data = new_data
    return data

In [54]:
def split_event_transitions(data, one_timestamp ):
    temp_raw = list()
    if one_timestamp:
        for event in data:
            temp_event = event.copy()
            temp_event['timestamp'] = temp_event.pop('end_timestamp')
            temp_event['event_type'] = 'complete'
            temp_raw.append(temp_event)
    else:
        for event in data:
            start_event = event.copy()
            complete_event = event.copy()
            start_event.pop('end_timestamp')
            complete_event.pop('start_timestamp')
            start_event['timestamp'] = start_event.pop('start_timestamp')
            complete_event['timestamp'] = complete_event.pop('end_timestamp')
            start_event['event_type'] = 'start'
            complete_event['event_type'] = 'complete'
            temp_raw.append(start_event)
            temp_raw.append(complete_event)
    data = temp_raw
    return data

In [56]:
log = pd.read_csv(os.path.join(input_data_folder, in_filename_csv), sep=",", dtype={'user': str})
one_timestamp = True
timeformat = '%Y-%m-%dT%H:%M:%S.%f'
column_names = {'Case ID': 'caseid',
                'Activity': 'task',
                'lifecycle:transition': 'event_type', #---#
                'Resource': 'user'}
filter_d_attrib = {
    'timeformat': '%Y-%m-%dT%H:%M:%S.%f',
    'column_names': column_names,
    'one_timestamp': one_timestamp,
    'ns_include': True}
if one_timestamp:
    column_names['Complete Timestamp'] = 'end_timestamp'
    log = log.rename(columns=column_names)
    log = log.astype({'caseid': object})
    log = (log[(log.task != 'Start') & (log.task != 'End')]
           .reset_index(drop=True))
    if filter_d_attrib:
        log = log[['caseid', 'task', 'user', 'end_timestamp']]
    log['end_timestamp'] = pd.to_datetime(log['end_timestamp'],
                                          format=timeformat)
else:
    column_names['Start Timestamp'] = 'start_timestamp'
    column_names['Complete Timestamp'] = 'end_timestamp'
    log = log.rename(columns=column_names)
    log = log.astype({'caseid': object})
    log = (log[(log.task != 'Start') & (log.task != 'End')]
           .reset_index(drop=True))
    if filter_d_attrib:
        log = log[['caseid', 'task', 'user',
                   'start_timestamp', 'end_timestamp']]
    log['start_timestamp'] = pd.to_datetime(log['start_timestamp'],
                                            format=timeformat)
    log['end_timestamp'] = pd.to_datetime(log['end_timestamp'],
                                          format=timeformat)
log['user'].fillna('SYS', inplace=True)
data = log.to_dict('records')
data = append_csv_start_end(data)
data = split_event_transitions(data, one_timestamp)
data = pd

In [58]:
print(data[:10])

[{'caseid': 'A', 'task': 'Start', 'user': 'Start', 'timestamp': Timestamp('2014-10-22 09:15:41+0000', tz='UTC'), 'event_type': 'complete'}, {'caseid': 'A', 'task': 'ER Registration', 'user': 'A', 'timestamp': Timestamp('2014-10-22 09:15:41+0000', tz='UTC'), 'event_type': 'complete'}, {'caseid': 'A', 'task': 'LacticAcid', 'user': 'B', 'timestamp': Timestamp('2014-10-22 09:27:00+0000', tz='UTC'), 'event_type': 'complete'}, {'caseid': 'A', 'task': 'Leucocytes', 'user': 'B', 'timestamp': Timestamp('2014-10-22 09:27:00+0000', tz='UTC'), 'event_type': 'complete'}, {'caseid': 'A', 'task': 'CRP', 'user': 'B', 'timestamp': Timestamp('2014-10-22 09:27:00+0000', tz='UTC'), 'event_type': 'complete'}, {'caseid': 'A', 'task': 'ER Triage', 'user': 'C', 'timestamp': Timestamp('2014-10-22 09:33:37+0000', tz='UTC'), 'event_type': 'complete'}, {'caseid': 'A', 'task': 'ER Sepsis Triage', 'user': 'A', 'timestamp': Timestamp('2014-10-22 09:34:00+0000', tz='UTC'), 'event_type': 'complete'}, {'caseid': 'A', '

In [35]:
print(data[:1])

[{'caseid': 'CNA', 'task': 'ER Registration', 'user': 'A', 'end_timestamp': Timestamp('2014-09-19 06:14:18+0000', tz='UTC')}]


In [59]:
data = pd.DataFrame.from_dict(data)

In [60]:
print(data[:5])

  caseid             task   user                 timestamp event_type
0      A            Start  Start 2014-10-22 09:15:41+00:00   complete
1      A  ER Registration      A 2014-10-22 09:15:41+00:00   complete
2      A       LacticAcid      B 2014-10-22 09:27:00+00:00   complete
3      A       Leucocytes      B 2014-10-22 09:27:00+00:00   complete
4      A              CRP      B 2014-10-22 09:27:00+00:00   complete


In [ ]:
path = os.path.join(self.output_route,
                    'parameters',
                    'model_parameters.json')

if 'activity' in data:
    del data['activity']
self.parms = {**self.parms, **{k: v for k, v in data.items()}}
self.parms['dim'] = {k: int(v) for k, v in data['dim'].items()}
with open(path) as file:
    filter_parms_data = json.load(file)
    filter_parms['index_ac'] = {int(k): v
                              for k, v in data['index_ac'].items()}
    filter_parms['index_rl'] = {int(k): v
                              for k, v in data['index_rl'].items()}
    file.close()
    self.ac_index = {v: k for k, v in self.parms['index_ac'].items()}
    self.rl_index = {v: k for k, v in self.parms['index_rl'].items()}

In [24]:
l = [[1,2,3,4],[5,6,7,8,9,10],[11,12,13]]

In [4]:
key = l.index([5,6,7,8,9,10])

In [5]:
print(key)

1


In [6]:
len(l)

3

In [7]:
dictA = {1:'Y', 2:'E', 3:'E'}
dictB = {"A":'Y', "B":'A', "C":'W'}

In [10]:
dictB["A"] in dictA

False

In [12]:
for name, age in dictA.items():
    if age == dictB["A"]:
        print(name)

1


In [25]:
l

[[1, 2, 3, 4], [5, 6, 7, 8, 9, 10], [11, 12, 13]]

In [27]:
l[1:]

[[5, 6, 7, 8, 9, 10], [11, 12, 13]]

In [28]:
z = [[0], [0, 4.0], [0, 4.0, 6.0], [0, 4.0, 6.0, 3.0], [0, 4.0, 6.0, 3.0, 10.0], [0, 4.0, 6.0, 3.0, 10.0, 5.0], [0, 4.0, 6.0, 3.0, 10.0, 5.0, 8.0], [0, 4.0, 6.0, 3.0, 10.0, 5.0, 8.0, 7.0], [0, 4.0, 6.0, 3.0, 10.0, 5.0, 8.0, 7.0, 12.0]]

In [29]:
z

[[0],
 [0, 4.0],
 [0, 4.0, 6.0],
 [0, 4.0, 6.0, 3.0],
 [0, 4.0, 6.0, 3.0, 10.0],
 [0, 4.0, 6.0, 3.0, 10.0, 5.0],
 [0, 4.0, 6.0, 3.0, 10.0, 5.0, 8.0],
 [0, 4.0, 6.0, 3.0, 10.0, 5.0, 8.0, 7.0],
 [0, 4.0, 6.0, 3.0, 10.0, 5.0, 8.0, 7.0, 12.0]]

In [36]:
z[:5]

[[0], [0, 4.0], [0, 4.0, 6.0], [0, 4.0, 6.0, 3.0], [0, 4.0, 6.0, 3.0, 10.0]]

In [37]:
import streamlit as st
from streamlit.hashing import _CodeHasher

try:
    # Before Streamlit 0.65
    from streamlit.ReportThread import get_report_ctx
    from streamlit.server.Server import Server
except ModuleNotFoundError:
    # After Streamlit 0.65
    from streamlit.report_thread import get_report_ctx
    from streamlit.server.server import Server


def main():
    state = _get_state()
    pages = {
        "Dashboard": page_dashboard,
        "Settings": page_settings,
    }

    st.sidebar.title(":floppy_disk: Page states")
    page = st.sidebar.radio("Select your page", tuple(pages.keys()))

    # Display the selected page with the session state
    pages[page](state)

    # Mandatory to avoid rollbacks with widgets, must be called at the end of your app
    state.sync()


def page_dashboard(state):
    st.title(":chart_with_upwards_trend: Dashboard page")
    display_state_values(state)


def page_settings(state):
    st.title(":wrench: Settings")
    display_state_values(state)

    st.write("---")
    options = ["Hello", "World", "Goodbye"]
    state.input = st.text_input("Set input value.", state.input or "")
    state.slider = st.slider("Set slider value.", 1, 10, state.slider)
    state.radio = st.radio("Set radio value.", options, options.index(state.radio) if state.radio else 0)
    state.checkbox = st.checkbox("Set checkbox value.", state.checkbox)
    state.selectbox = st.selectbox("Select value.", options, options.index(state.selectbox) if state.selectbox else 0)
    state.multiselect = st.multiselect("Select value(s).", options, state.multiselect)

    # Dynamic state assignments
    for i in range(3):
        key = f"State value {i}"
        state[key] = st.slider(f"Set value {i}", 1, 10, state[key])


def display_state_values(state):
    st.write("Input state:", state.input)
    st.write("Slider state:", state.slider)
    st.write("Radio state:", state.radio)
    st.write("Checkbox state:", state.checkbox)
    st.write("Selectbox state:", state.selectbox)
    st.write("Multiselect state:", state.multiselect)
    
    for i in range(3):
        st.write(f"Value {i}:", state[f"State value {i}"])

    if st.button("Clear state"):
        state.clear()


class _SessionState:

    def __init__(self, session, hash_funcs):
        """Initialize SessionState instance."""
        self.__dict__["_state"] = {
            "data": {},
            "hash": None,
            "hasher": _CodeHasher(hash_funcs),
            "is_rerun": False,
            "session": session,
        }

    def __call__(self, **kwargs):
        """Initialize state data once."""
        for item, value in kwargs.items():
            if item not in self._state["data"]:
                self._state["data"][item] = value

    def __getitem__(self, item):
        """Return a saved state value, None if item is undefined."""
        return self._state["data"].get(item, None)
        
    def __getattr__(self, item):
        """Return a saved state value, None if item is undefined."""
        return self._state["data"].get(item, None)

    def __setitem__(self, item, value):
        """Set state value."""
        self._state["data"][item] = value

    def __setattr__(self, item, value):
        """Set state value."""
        self._state["data"][item] = value
    
    def clear(self):
        """Clear session state and request a rerun."""
        self._state["data"].clear()
        self._state["session"].request_rerun()
    
    def sync(self):
        """Rerun the app with all state values up to date from the beginning to fix rollbacks."""

        # Ensure to rerun only once to avoid infinite loops
        # caused by a constantly changing state value at each run.
        #
        # Example: state.value += 1
        if self._state["is_rerun"]:
            self._state["is_rerun"] = False
        
        elif self._state["hash"] is not None:
            if self._state["hash"] != self._state["hasher"].to_bytes(self._state["data"], None):
                self._state["is_rerun"] = True
                self._state["session"].request_rerun()

        self._state["hash"] = self._state["hasher"].to_bytes(self._state["data"], None)


def _get_session():
    session_id = get_report_ctx().session_id
    session_info = Server.get_current()._get_session_info(session_id)

    if session_info is None:
        raise RuntimeError("Couldn't get your Streamlit Session object.")
    
    return session_info.session


def _get_state(hash_funcs=None):
    session = _get_session()

    if not hasattr(session, "_custom_session_state"):
        session._custom_session_state = _SessionState(session, hash_funcs)

    return session._custom_session_state


if __name__ == "__main__":
    main()

AttributeError: 'NoneType' object has no attribute 'session_id'

In [40]:
n = [0]

In [41]:
n = n + 1

TypeError: can only concatenate list (not "int") to list

In [45]:
u = 4

In [46]:
y = [u-1]

In [47]:
y

[3]